#Generative Adversarial Network (GAN)

Código base:
https://github.com/eriklindernoren/Keras-GAN



In [ ]:
#Carregar bibliotecas
from keras.layers import Input, Dense, Reshape, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import random
# Seed para reprodução de resultados
seed = 10
random.seed(seed)
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
input_shape = (500,)
latent_dim = 100

In [ ]:
#gerar dados de entrada com distribuição uniforme
import pandas as pd
X = pd.DataFrame(np.random.uniform(0,1,size=(10000, latent_dim)))
X.head()

In [ ]:
#distribuicao para amostra
linha1 = X.iloc[1,:]
histogram_i = plt.hist(linha1)

###Exercício 1: Gerar dados com distbuição normal. Salvar em uma variável chamada 'y'. Plotar o histograma. Imprimir a média e o desvio de uma amostra qualquer.

In [ ]:
# Criar o Gerador
generator = Sequential()

generator.add(Dense(256, input_dim=latent_dim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(np.prod(input_shape), activation='linear'))
generator.add(Reshape(input_shape))
generator.summary()

noise = Input(shape=(latent_dim,))
distribution = generator(noise)
generator = Model(noise, distribution)

# Compilar o gerador
optimizer = Adam(0.0002, 0.5)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
# Criar o Discriminador
discriminator = Sequential()

discriminator.add(Dense(512, input_shape=input_shape))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))

discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

distribution = Input(shape=input_shape)
validity = discriminator(distribution)
discriminator = Model(distribution, validity)

# Compilar o discriminador
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# O gerador recebe um ruído como entrada e gera a amostra com a distribuição aprendida
input = Input(shape=(latent_dim,))
gen_data = generator(input)
validity = discriminator(gen_data)

# Para o modelo combinado, somente treinaremos o gerador
discriminator.trainable = False

# Modelo combinado: stacked generator and discriminator
# Treina o gerador para enganar o discriminador
combined = Model(input, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Mostrar a estrutura do modelo criado
config = combined.get_config()
model = Model.from_config(config)
model.summary()

In [ ]:
def sample_images(epoch):
  r, c = 5, 5
  noise = np.random.uniform(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].hist(gen_imgs[cnt, :])
          axs[i,j].axis('off')
          # axs[i,j].set_title(epoch)
          fig.suptitle(epoch)
          cnt += 1
  plt.subplots(r, c)
  plt.close()

In [ ]:
batch_size=32
epochs=1200
sample_interval=200

# history
d_loss_history = []
d_acc_history = []
d_acc_real_history = []
d_acc_fake_history = []
g_loss_history = []
g_acc_history = []

# Vetores com 0s e 1s indicando, respectivamente, amostras fakes e reais
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

# ---------------------
#  Treina o Discriminador
# ---------------------
# Seleciona um batch aleatório de dados
  idx = np.random.randint(0, len(X[1]), batch_size)
  imgs = y.iloc[idx,:]
  noise = X.iloc[idx,:]

  # Gera um batch de novos dados
  gen_imgs = generator.predict(noise)

  # Treina o discriminador
  d_loss_real = discriminator.train_on_batch(imgs, valid)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  d_loss_history.append(d_loss[0])
  d_acc_history.append(d_loss[1])
  d_acc_real_history.append(d_loss_real[1])
  d_acc_fake_history.append(d_loss_fake[1])

  # ---------------------
  #  Treina o Gerador
  # ---------------------

  # Treina o gerador para que faça o discriminador classificar os dados criados como reais
  g_loss, g_acc = combined.train_on_batch(noise, valid)

  g_loss_history.append(g_loss)
  g_acc_history.append(g_acc)
  
  # Progresso
  print ("%d [D loss: %f, accDreal.: %.2f%%, accDfake.: %.2f%%] [G loss: %f, accG.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss, 100*g_acc))

  # Gerar imagens ao longo do treinamento
  if epoch % sample_interval == 0:
    sample_images(epoch)


In [ ]:
# Gráfico da evolução do erro e da acurácia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4))
ax1.plot(d_loss_history, label='discriminador')
ax1.plot(g_loss_history, label='gerador')
ax1.set_title('Loss')
ax1.set_xlabel('época')
ax1.set_ylabel('loss')
ax1.legend()

ax2.plot(d_acc_history, label='discriminador')
ax2.plot(g_acc_history, label='gerador')
ax2.set_title('Acurácia')
ax2.set_xlabel('época')
ax2.set_ylabel('acurácia')
ax2.legend()
plt.show()

In [ ]:
plt.plot(d_acc_real_history, label='real')
plt.plot(d_acc_fake_history, label='fake')
plt.title('Acurácia Discriminador')
plt.xlabel('época')
plt.ylabel('acurácia')
plt.legend()
plt.show()

###Exercício 2: Criar uma amostra para teste. Visualizar a base (função head())

###Exercício 3: visualizar algumas características da base através da função describe()

In [ ]:
#por coluna. Aproximadamente a mesma distribuiçao que criamos por linha


###Exercicio 4: plotar o histograma da base de teste (ruído) gerada

In [ ]:
#distribuicao: primeira linha

###Exercício 5: Utilizar a GAN treinada para gerar dados com a distribuição desejada

###Exercício 6: plotar o histograma da distribuição dos dados gerados pela GAN